<a href="https://colab.research.google.com/github/AprilisMaijs/LV-twitter-sentiment/blob/main/hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install azure-ai-formrecognizer==3.2.0
! pip install azure-core
! python -m pip install azure-ai-vision
! wget -O - https://www.openssl.org/source/openssl-1.1.1u.tar.gz | tar zxf -
! cd openssl-1.1.1u
! ./config --prefix=/usr/local
! make -j $(nproc)
! sudo make install_sw install_ssldirs
! sudo ldconfig -v
! export SSL_CERT_DIR=/etc/ssl/certs
! cd -
! sudo apt-get install build-essential libssl-dev wget

--2023-11-30 20:13:03--  https://www.openssl.org/source/openssl-1.1.1u.tar.gz
Resolving www.openssl.org (www.openssl.org)... 23.8.72.51, 2600:1407:3c00:1089::c1e, 2600:1407:3c00:1090::c1e
Connecting to www.openssl.org (www.openssl.org)|23.8.72.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9892176 (9.4M) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>]   9.43M  20.6MB/s    in 0.5s    

2023-11-30 20:13:04 (20.6 MB/s) - written to stdout [9892176/9892176]

/bin/bash: line 1: ./config: No such file or directory
make: *** No targets specified and no makefile found.  Stop.
make: *** No rule to make target 'install_sw'.  Stop.
/sbin/ldconfig.real: Path `/usr/local/cuda-11/targets/x86_64-linux/lib' given more than once
(from /etc/ld.so.conf.d/989_cuda-11.conf:1 and /etc/ld.so.conf.d/000_cuda.conf:1)
/sbin/ldconfig.real: Path `/usr/local/cuda-11.8/targets/x86_64-linux/lib' given more than once
(from /etc/ld.so.conf.d/gds-11

In [2]:
import os
import azure.ai.vision as sdk
from statistics import mean
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

In [4]:
# Replace these variables with your own values
endpoint = "https://robocoptest.cognitiveservices.azure.com/"
key = "7af3731a63e642be8e13c6e57c9808e3"
model_id = "trained_model_label1"

# Create FormRecognizerClient
document_analysis_client = DocumentAnalysisClient(endpoint, AzureKeyCredential(key))

# Replace 'image_path' with the path to your image file
image_path = "/content/MicrosoftTeams-image.png"

# Open the image file
with open(image_path, "rb") as f:
    # Pass the image to the model for analysis
    poller = document_analysis_client.begin_analyze_document(
           model_id, document=f, locale="en-US")
# Get the result
result = poller.result()

# Process the result as needed


store the value of each field & confidence


In [5]:
for idx, label in enumerate(result.documents):
    print(label.fields.get("reason").value, label.fields.get("reason").confidence)

    fieldValues = dict(
        case_client_reference = label.fields.get("case_client_reference").value,
        case_client_reference_confidence = label.fields.get("case_client_reference").confidence,
        seal_client_reference = label.fields.get("seal_client_reference").value,
        seal_client_reference_confidence = label.fields.get("seal_client_reference").confidence,
        client_reference_service = label.fields.get("client_reference_service").value,
        client_reference_service_confidence = label.fields.get("client_reference_service").confidence,
        reason = label.fields.get("reason").value,
        reason_confidence = label.fields.get("reason").confidence,
        description = label.fields.get("description").value,
        description_confidence = label.fields.get("description").confidence,
        DNA_identification_code = label.fields.get("DNA_identification_code").value,
        DNA_identification_code_confidence = label.fields.get("DNA_identification_code").confidence
        )
print(fieldValues)


meurtre 0.944
{'case_client_reference': '2021-999', 'case_client_reference_confidence': 0.993, 'seal_client_reference': '05-DNA', 'seal_client_reference_confidence': 0.958, 'client_reference_service': 'COB Thoiry', 'client_reference_service_confidence': 0.99, 'reason': 'meurtre', 'reason_confidence': 0.944, 'description': 'crachat de la victime', 'description_confidence': 0.99, 'DNA_identification_code': 650476501.0, 'DNA_identification_code_confidence': 0.995}


In [ ]:
# OCR Function
class do_ocr:
  def __init__(self, formUrl, model='trained_model_label1'):
    self.formUrl = formUrl
    self.model = model
  def analyze_read(self):
      document_analysis_client = DocumentAnalysisClient(endpoint=endpoint, credential=AzureKeyCredential(key))
      poller = document_analysis_client.begin_analyze_document_from_url(self.model, self.formUrl)
      result = poller.result()
      dict_page = {}
      word_confidence = []
      for idx, style in enumerate(result.styles):
          print("Document contains {} content".format("handwritten" if style.is_handwritten else "no handwritten"))
      for page in result.pages:
          dict_page['page_number'] = {page.page_number}
          for word in page.words:
            word_confidence.append({word.content:word.confidence})
          dict_page['content'] = word_confidence
      return result.content, dict_page
  def preprocess_results(self):
      content, predictions = self.analyze_read()
      average_score = mean(value for dicts in predictions.get('content', []) for value in dicts.values())
      final_dict = {key:value for dicts in predictions.get('content', []) for key, value in dicts.items() if value < average_score}
      return content, average_score, final_dict

if __name__ == "__main__":
    instance = do_ocr(formUrl).preprocess_results()
    content, average_score, final_dict = instance

Document contains handwritten content
Document contains handwritten content
Document contains handwritten content


In [ ]:
# Output

#content
#average_score
final_dict

{'O2-DNA': 0.827,
 'aggravé': 0.863,
 '1500': 0.746,
 'bla': 0.643,
 ':': 0.937,
 'couteau': 0.808,
 'ensanglanté': 0.642,
 'DSang': 0.847,
 '-': 0.003,
 'Autre': 0.886,
 '856478501': 0.621,
 'Signature)|': 0.622}

In [ ]:
VISION_KEY=""
VISION_ENDPOINT="https://objectclassify.cognitiveservices.azure.com/"
url="https://learn.microsoft.com/azure/ai-services/computer-vision/media/quickstarts/presentation.png"
service_options = sdk.VisionServiceOptions(VISION_ENDPOINT,VISION_KEY)

In [ ]:
# OBject Classification

class do_classification:
  def __init__(self,url,service_options,language='en'):
    self.url = url
    self.language = language
    self.service_options = service_options
  def do_analysis(self):
    vision_source = sdk.VisionSource(url=self.url)
    analysis_options = sdk.ImageAnalysisOptions()
    analysis_options.features = (sdk.ImageAnalysisFeature.CAPTION |sdk.ImageAnalysisFeature.TEXT)
    analysis_options.language = self.language
    analysis_options.gender_neutral_caption = True
    image_analyzer = sdk.ImageAnalyzer(self.service_options, vision_source, analysis_options)
    result = image_analyzer.analyze()
    return result
  def do_preprocessing(self):
    result = self.do_analysis()
    data_dict = {}
    line_dict = {}
    if result.reason == sdk.ImageAnalysisResultReason.ANALYZED:
        if result.caption is not None:
          data_dict['Caption'] = {result.caption.content:result.caption.confidence}
        if result.text is not None:
            for line in result.text.lines:
                for word in line.words:
                  line_dict[line.content] = {word.content:word.confidence}
        return data_dict, line_dict
    else:
        error_details = sdk.ImageAnalysisErrorDetails.from_result(result)
        print(" Analysis failed.")
        print("   Error reason: {}".format(error_details.reason))
        print("   Error code: {}".format(error_details.error_code))
        print("   Error message: {}".format(error_details.message))
        return error_details, _

In [ ]:
instance = do_classification(url=url,service_options=sdk.VisionServiceOptions(VISION_ENDPOINT,VISION_KEY)).do_preprocessing()
data_dict, line_dict = instance
sentence = []
for key, value in line_dict.items():
  sentence.append(key)

complete_sentence = ' '.join(sentence)

In [ ]:
# Output
complete_sentence
data_dict

Reference:
1. [Use Document Intelligence models](https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/how-to-guides/use-sdk-rest-api?view=doc-intel-4.0.0&tabs=linux&pivots=programming-language-python)
2. [Quickstart: Image Analysis 4.0](https://learn.microsoft.com/en-us/azure/ai-services/computer-vision/quickstarts-sdk/image-analysis-client-library-40?tabs=visual-studio%2Clinux&pivots=programming-language-python)